In [40]:
# All Includes

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import json
import glob
import os
#import tensorflow as tf  # Version 1.0.0 (some previous versions are used in past commits)
from sklearn import metrics
import tensorflow.compat.v1 as tf
from tensorflow.python.ops import rnn_cell_impl
tf.disable_v2_behavior()

In [41]:
# # Note: Linux bash commands start with a "!" inside those "ipython notebook" cells

# DATA_PATH = "data/"

# !pwd && ls
# os.chdir(DATA_PATH)
# !pwd && ls

# !python download_dataset.py

# !pwd && ls
# os.chdir("..")
# !pwd && ls

# DATASET_PATH = DATA_PATH + "UCI HAR Dataset/"
# print("\n" + "Dataset is now located at: " + DATASET_PATH)

# Preparing Data

In [42]:
# Useful Constants

# Those are separate normalised input features for the neural network
INPUT_SIGNAL_TYPES = [
    "body_acc_x_",
    "body_acc_y_",
    "body_acc_z_",
    "body_gyro_x_",
    "body_gyro_y_",
    "body_gyro_z_",
    "total_acc_x_",
    "total_acc_y_",
    "total_acc_z_"
]

# Output classes to learn how to classify
LABELS = [
    "WALKING",
    "WALKING_UPSTAIRS",
    "WALKING_DOWNSTAIRS",
    "SITTING",
    "STANDING",
    "LAYING"
]

DATASET_PATH = "UCI HAR Dataset/"

TRAIN = "train/"
TEST = "test/"


# Load "X" (the neural network's training and testing inputs)
def load_X(X_signals_paths):
    X_signals = []

    for signal_type_path in X_signals_paths:
        file = open(signal_type_path, 'r')
        # Read dataset from disk, dealing with text files' syntax
        X_signals.append(
            [np.array(serie, dtype=np.float32) for serie in [
                row.replace('  ', ' ').strip().split(' ') for row in file
            ]]
        )
        file.close()

    return np.transpose(np.array(X_signals), (1, 2, 0))

X_train_signals_paths = [
    DATASET_PATH + TRAIN + "Inertial Signals/" + signal + "train.txt" for signal in INPUT_SIGNAL_TYPES
]
X_test_signals_paths = [
    DATASET_PATH + TEST + "Inertial Signals/" + signal + "test.txt" for signal in INPUT_SIGNAL_TYPES
]

X_train = load_X(X_train_signals_paths)
X_test = load_X(X_test_signals_paths)


# Load "y" (the neural network's training and testing outputs)
def load_y(y_path):
    file = open(y_path, 'r')

    # Read dataset from disk, dealing with text file's syntax
    # y_ temporary variable for raw data
    y_ = np.array(
        [elem for elem in [
            row.replace('  ', ' ').strip().split(' ') for row in file
        ]],
        dtype=np.int32
    )
    file.close()

    # Converts to 0-indexed (0-5) for neural network
    return y_ - 1

y_train_path = DATASET_PATH + TRAIN + "y_train.txt"
y_test_path = DATASET_PATH + TEST + "y_test.txt"

y_train = load_y(y_train_path)
y_test = load_y(y_test_path)

# Parameters

In [43]:
# Input Data

training_data_count = len(X_train)  # 7352 training series (with 50% overlap between each serie)
test_data_count = len(X_test)  # 2947 testing series
n_steps = len(X_train[0])  # First sample (128 timesteps per sample)
n_input = len(X_train[0][0])  # First timestep of first sample (9 input parameters per timestep)


# LSTM Neural Network's internal structure

n_hidden = 32  # Try: 32 (default), 64, 128, or 256
n_classes = 6 # Total classes (should go up, or should go down)


# Training

learning_rate = 0.0050
lambda_loss_amount = 0.002 #0.0015
training_iters = training_data_count * 500  # 300(default)
batch_size = 1500
display_iter = 30000  # To show test set accuracy during training


# Some debugging info

print("Some useful info to get an insight on dataset's shape and normalisation:")
print("(X shape, y shape, every X's mean, every X's standard deviation)")
print(X_test.shape, y_test.shape, np.mean(X_test), np.std(X_test))
print("The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.")


Some useful info to get an insight on dataset's shape and normalisation:
(X shape, y shape, every X's mean, every X's standard deviation)
(2947, 128, 9) (2947, 1) 0.09913992 0.39567086
The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.


# Functions

In [44]:

def LSTM_RNN(_X, _weights, _biases):
    # Function returns a tensorflow LSTM (RNN) artificial neural network from given parameters.
    # Moreover, two LSTM cells are stacked which adds deepness to the neural network.
    # Note, some code of this notebook is inspired from an slightly different
    # RNN architecture used on another dataset, some of the credits goes to
    # "aymericdamien" under the MIT license.

    # (NOTE: This step could be greatly optimised by shaping the dataset once
    # input shape: (batch_size, n_steps, n_input)
    _X = tf.transpose(_X, [1, 0, 2])  # permute n_steps and batch_size
    # Reshape to prepare input to hidden activation
    _X = tf.reshape(_X, [-1, n_input])
    # new shape: (n_steps*batch_size, n_input)

    _X = tf.nn.relu(tf.matmul(_X, _weights['hidden']) + _biases['hidden'])
    # Split data because rnn cell needs a list of inputs for the RNN inner loop
    _X = tf.split(_X, n_steps, 0)
    # new shape: n_steps * (batch_size, n_hidden)

    # Define two stacked LSTM cells (two recurrent layers deep) with tensorflow
    # Using rnn_cell_impl for TF 2.x compatibility (tf.contrib is removed in TF 2.x)
    lstm_cell_1 = rnn_cell_impl.LSTMCell(n_hidden, forget_bias=1.0)
    lstm_cell_2 = rnn_cell_impl.LSTMCell(n_hidden, forget_bias=1.0)
    lstm_cells = rnn_cell_impl.MultiRNNCell([lstm_cell_1, lstm_cell_2])
    # Get LSTM cell output
    outputs, states = tf.nn.static_rnn(lstm_cells, _X, dtype=tf.float32)

    # Get last time step's output feature for a "many-to-one" style classifier,
    # as in the image describing RNNs at the top of this page
    lstm_last_output = outputs[-1]

    # Linear activation
    return tf.matmul(lstm_last_output, _weights['out']) + _biases['out']


def extract_batch_size(_train, step, batch_size):
    # Function to fetch a "batch_size" amount of data from "(X|y)_train" data.
    shape = list(_train.shape)
    shape[0] = batch_size
    batch_s = np.empty(shape)

    for i in range(batch_size):
        # Loop index
        index = ((step-1)*batch_size + i) % len(_train)
        batch_s[i] = _train[index]

    return batch_s


def one_hot(y_, n_classes=n_classes):
    # Function to encode neural one-hot output labels from number indexes
    # e.g.:
    # one_hot(y_=[[5], [0], [3]], n_classes=6):
    #     return [[0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0]]

    y_ = y_.reshape(len(y_))    
    return np.eye(n_classes)[np.array(y_, dtype=np.int32)]  # Returns FLOATS

# Build the NN

In [45]:
tf.reset_default_graph()

# Graph input/output
x = tf.placeholder(tf.float32, [None, n_steps, n_input], name='input_x')
y = tf.placeholder(tf.float32, [None, n_classes], name='input_y')

# Graph weights - EXPLICITLY NAMED for reliable saving/restoring
weights = {
    'hidden': tf.Variable(tf.random_normal([n_input, n_hidden]), name='weights_hidden'), # Hidden layer weights
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes], mean=1.0), name='weights_out')
}
biases = {
    'hidden': tf.Variable(tf.random_normal([n_hidden]), name='biases_hidden'),
    'out': tf.Variable(tf.random_normal([n_classes]), name='biases_out')
}

pred = LSTM_RNN(x, weights, biases)

# Loss, optimizer and evaluation
l2 = lambda_loss_amount * sum(
    tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables()
) # L2 loss prevents this overkill neural network to overfit the data
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred)) + l2 # Softmax loss
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost) # Adam Optimizer

correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Train the NN

In [46]:

# To keep track of training's performance
test_losses = []
test_accuracies = []
train_losses = []
train_accuracies = []

# Launch the graph
sess = tf.InteractiveSession(config=tf.ConfigProto(log_device_placement=True))
init = tf.global_variables_initializer()
sess.run(init)

# Perform Training steps with "batch_size" amount of example data at each loop
step = 1
while step * batch_size <= training_iters:
    batch_xs = extract_batch_size(X_train, step, batch_size)
    batch_ys = one_hot(extract_batch_size(y_train, step, batch_size))

    # Fit training using batch data
    _, loss, acc = sess.run(
        [optimizer, cost, accuracy],
        feed_dict={
            x: batch_xs,
            y: batch_ys
        }
    )
    train_losses.append(loss)
    train_accuracies.append(acc)

    # Evaluate network only at some steps for faster training:
    if (step*batch_size % display_iter == 0) or (step == 1) or (step * batch_size > training_iters):

        # To not spam console, show training accuracy/loss in this "if"
        print("Training iter #" + str(step*batch_size) + \
              ":   Batch Loss = " + "{:.6f}".format(loss) + \
              ", Accuracy = {}".format(acc))

        # Evaluation on the test set (no learning made here - just evaluation for diagnosis)
        loss, acc = sess.run(
            [cost, accuracy],
            feed_dict={
                x: X_test,
                y: one_hot(y_test)
            }
        )
        test_losses.append(loss)
        test_accuracies.append(acc)
        print("PERFORMANCE ON TEST SET: " + \
              "Batch Loss = {}".format(loss) + \
              ", Accuracy = {}".format(acc))

    step += 1

print("Optimization Finished!")

# Accuracy for test data
one_hot_predictions, accuracy, final_loss = sess.run(
    [pred, accuracy, cost], # Note: pred returns the raw logits (scores before softmax), not probabilities.
    feed_dict={
        x: X_test,
        y: one_hot(y_test)
    }
)

test_losses.append(final_loss)
test_accuracies.append(accuracy)

print("FINAL RESULT: " + \
      "Batch Loss = {}".format(final_loss) + \
      ", Accuracy = {}".format(accuracy))


Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5



ERROR:tensorflow:An interactive session is already active. This can cause out-of-memory errors or some other unexpected errors (due to the unpredictable timing of garbage collection) in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s). Please use `tf.Session()` if you intend to productionize.


Training iter #1500:   Batch Loss = 3.579559, Accuracy = 0.1393333375453949
PERFORMANCE ON TEST SET: Batch Loss = 2.677802324295044, Accuracy = 0.36851033568382263
Training iter #30000:   Batch Loss = 1.614421, Accuracy = 0.6473333239555359
PERFORMANCE ON TEST SET: Batch Loss = 1.7348181009292603, Accuracy = 0.6196131706237793
Training iter #60000:   Batch Loss = 1.393216, Accuracy = 0.7566666603088379
PERFORMANCE ON TEST SET: Batch Loss = 1.505052089691162, Accuracy = 0.734645426273346
Training iter #90000:   Batch Loss = 1.213658, Accuracy = 0.809333324432373
PERFORMANCE ON TEST SET: Batch Loss = 1.4393060207366943, Accuracy = 0.7712928652763367
Training iter #120000:   Batch Loss = 1.055738, Accuracy = 0.878000020980835
PERFORMANCE ON TEST SET: Batch Loss = 1.409346103668213, Accuracy = 0.7617916464805603
Training iter #150000:   Batch Loss = 0.993551, Accuracy = 0.8939999938011169
PERFORMANCE ON TEST SET: Batch Loss = 1.2538471221923828, Accuracy = 0.8235493898391724
Training iter 

In [ ]:
#sess.close()

# Save the Model

In [51]:
# Save the trained model
saver = tf.train.Saver()

# # Create model directory if it doesn't exist
# os.makedirs('./model', exist_ok=True)

# # Save model to disk
# save_path = saver.save(sess, "./model/lstm_model.ckpt")
# print(f"Model saved to: {save_path}")

# Find next available version number
# existing = glob.glob('./model/lstm_model_v*')
# version = len(existing) + 1
# model_dir = f'./model/lstm_model_v{version:03d}'
# Not saved at the same server, needs to save manuelly

model_dir = f'./model/lstm_model_v002'
os.makedirs(model_dir, exist_ok=True)
save_path = saver.save(sess, f"{model_dir}/model.ckpt")
print(f"Model saved to: {save_path}")


# Save metadata
model_info = {
    'n_hidden': n_hidden,
    'n_classes': n_classes,
    'n_steps': n_steps,
    'n_input': n_input,
    'learning_rate': learning_rate,
    'lambda_loss_amount': lambda_loss_amount,
    'final_accuracy': float(accuracy),
    'final_loss': float(final_loss)
}

with open(f'{model_dir}/model_info.json', 'w') as f:
    json.dump(model_info, f, indent=4)
    
print(f"Model metadata saved to: {model_dir}/model_info.json")
print(f"\n✓ Model saved successfully!")
print(f"  - Accuracy: {accuracy*100:.2f}%")
print(f"  - Loss: {final_loss:.4f}")

#sess.close()

Model saved to: ./model/lstm_model_v002/model.ckpt
Model metadata saved to: ./model/lstm_model_v002/model_info.json

✓ Model saved successfully!
  - Accuracy: 91.18%
  - Loss: 0.3406


# Plot the training

In [49]:
# # (Inline plots: )
# %matplotlib inline

# font = {
#     'family' : 'Bitstream Vera Sans',
#     'weight' : 'bold',
#     'size'   : 18
# }
# matplotlib.rc('font', **font)
# ## Is equivalent to:
# #matplotlib.rc('font', family='Bitstream Vera Sans', weight='bold', size=18)

# plt.figure(figsize=(12, 12))

# indep_train_axis = np.array(range(batch_size, (len(train_losses)+1)*batch_size, batch_size))
# plt.plot(indep_train_axis, np.array(train_losses),     "b--", label="Train losses")
# plt.plot(indep_train_axis, np.array(train_accuracies), "g--", label="Train accuracies")

# indep_test_axis = np.append(
#     np.array(range(batch_size, len(test_losses)*display_iter, display_iter)[:-1]),
#     [training_iters]
# )
# plt.plot(indep_test_axis, np.array(test_losses),     "b-", label="Test losses")
# plt.plot(indep_test_axis, np.array(test_accuracies), "g-", label="Test accuracies")

# plt.title("Training session's progress over iterations")
# plt.legend(loc='upper right', shadow=True)
# plt.ylabel('Training Progress (Loss or Accuracy values)')
# plt.xlabel('Training iteration')

# plt.show()

# Confusion matrix 

In [50]:
# # Results

# predictions = one_hot_predictions.argmax(1)

# print("Testing Accuracy: {}%".format(100*accuracy))

# print("")
# print("Precision: {}%".format(100*metrics.precision_score(y_test, predictions, average="weighted")))
# print("Recall: {}%".format(100*metrics.recall_score(y_test, predictions, average="weighted")))
# print("f1_score: {}%".format(100*metrics.f1_score(y_test, predictions, average="weighted")))

# print("")
# print("Confusion Matrix:")
# confusion_matrix = metrics.confusion_matrix(y_test, predictions)
# print(confusion_matrix)
# normalised_confusion_matrix = np.array(confusion_matrix, dtype=np.float32)/np.sum(confusion_matrix)*100

# print("")
# print("Confusion matrix (normalised to % of total test data):")
# print(normalised_confusion_matrix)
# print("Note: training and testing data is not equally distributed amongst classes, ")
# print("so it is normal that more than a 6th of the data is correctly classifier in the last category.")

# # Plot Results:
# width = 12
# height = 12
# plt.figure(figsize=(width, height))
# plt.imshow(
#     normalised_confusion_matrix,
#     interpolation='nearest',
#     cmap=plt.cm.rainbow
# )
# plt.title("Confusion matrix \n(normalised to % of total test data)")
# plt.colorbar()
# tick_marks = np.arange(n_classes)
# plt.xticks(tick_marks, LABELS, rotation=90)
# plt.yticks(tick_marks, LABELS)
# plt.tight_layout()
# plt.ylabel('True label')
# plt.xlabel('Predicted label')
# plt.show()